In [1]:
import pandas as pd
import test_groups as TG

Tests = []

In [2]:
class PrepTests:
    """ Save testname and make a dataframe of test results.
    """
    def __init__(self,name):
        self.name = name
        self.dataframe = self.make_df()

    def make_df(self,add = False, dta =[]):
        if add:
            self.dataframe = pd.concat([self.dataframe, dta], axis= 1, ignore_index= True)
            return
        else:
            df = pd.DataFrame()
            return df

In [5]:
def DataPrep(data):
    for index, element in data.iterrows():
        strings = 0
        if isinstance(element["Sonuç"], str) and element["Sonuç"] != "None":
            strings += 1
            tn = f"Test {element['Test Adı']}"
            tablename = element['Test Adı']
            #tablename= data.iloc[index,0]
            tn =  PrepTests(tablename)
            Tests.append(tn)
        else:
            Tests[-1].make_df(add = True, dta = data.iloc[index])

    for test in Tests:
        test.dataframe = test.dataframe.transpose()

In [6]:
data = pd.read_excel("Sample.xlsx", header=0)
#data = data.drop(columns= "Referans Değer")
DataPrep(data)

In [7]:
data.head()

,Test Adı,Sonuç,Birim,Referans Değer
0,NMR Lipoporotein Profili,Sonuçlar için alt parametrelere bakınız. Teste...,NaN,NaN
1,Trigliserid,46,mg/dL,<150 Optimal Düzey >150 Artmış Kardiovasküler ...
2,"Kolestrol, total",214,mg/dL,<200 Optimal Düzey >200 Artmış Kardiovasküler ...
3,"Kolestrol, LDL",118,mg/dL,<70 Kardiyovasküler Hastalığı Olanlarda Optima...
4,"Kolestrol, HDL",88.5,mg/dL,<50 Optimal Düzey >50 Artmış Kardiovasküler Ha...


In [ ]:
data.loc[data["Test Adı"] == "Trigliserid", "Referans Değer"]